# Train and test split

In [8]:
test_images_names_list = [    
    "GF2_PMS1__L1A0001064454-MSS1",
    "GF2_PMS1__L1A0001118839-MSS1",
    "GF2_PMS1__L1A0001344822-MSS1",
    "GF2_PMS1__L1A0001348919-MSS1",
    "GF2_PMS1__L1A0001366278-MSS1",
    "GF2_PMS1__L1A0001366284-MSS1",
    "GF2_PMS1__L1A0001395956-MSS1",
    "GF2_PMS1__L1A0001432972-MSS1",
    "GF2_PMS1__L1A0001670888-MSS1",
    "GF2_PMS1__L1A0001680857-MSS1",
    "GF2_PMS1__L1A0001680858-MSS1",
    "GF2_PMS1__L1A0001757429-MSS1",
    "GF2_PMS1__L1A0001765574-MSS1",
    "GF2_PMS2__L1A0000607677-MSS2",
    "GF2_PMS2__L1A0000607681-MSS2",
    "GF2_PMS2__L1A0000718813-MSS2",
    "GF2_PMS2__L1A0001038935-MSS2",
    "GF2_PMS2__L1A0001038936-MSS2",
    "GF2_PMS2__L1A0001119060-MSS2",
    "GF2_PMS2__L1A0001367840-MSS2",
    "GF2_PMS2__L1A0001378491-MSS2",
    "GF2_PMS2__L1A0001378501-MSS2",
    "GF2_PMS2__L1A0001396036-MSS2",
    "GF2_PMS2__L1A0001396037-MSS2",
    "GF2_PMS2__L1A0001416129-MSS2",
    "GF2_PMS2__L1A0001471436-MSS2",
    "GF2_PMS2__L1A0001517494-MSS2",
    "GF2_PMS2__L1A0001591676-MSS2",
    "GF2_PMS2__L1A0001787564-MSS2",
    "GF2_PMS2__L1A0001821754-MSS2"
    ]



import os
def train_test_split_gaofen_images(images_dir,destiny_path_image_train,destiny_path_image_test):
    for image_idx, image_filename in enumerate(os.listdir(images_dir)):
        image_filename_cmp = image_filename.split(".")[0]
        if image_filename_cmp not in test_images_names_list:
            print(f"{image_idx}-{image_filename} is train")
            os.system("cp " + images_dir + image_filename + " " + destiny_path_image_train)
        else:
            print(f"{image_idx}-{image_filename} is test")
            os.system("cp " + images_dir + image_filename + " " + destiny_path_image_test)




def train_test_split_gaofen_masks(masks_dir,destiny_path_mask_train,destiny_path_mask_test):
    for mask_idx, mask_filename in enumerate(os.listdir(masks_dir)):
        mask_filename_cmp = mask_filename.split(".")[0]
        if mask_filename_cmp[:-8] not in test_images_names_list:
            print(f"{mask_idx}-{mask_filename} is train")
            os.system("cp " + masks_dir + mask_filename + " " + destiny_path_mask_train)
        else:
            print(f"{mask_idx}-{mask_filename} is test")
            os.system("cp " + masks_dir + mask_filename + " " + destiny_path_mask_test)



In [ ]:
#IMAGES
images_dir = "" #ADD PATH
destiny_path_image_train = ""#ADD PATH
destiny_path_image_test = ""#ADD PATH

train_test_split_gaofen_images(images_dir,destiny_path_image_train,destiny_path_image_test)

In [ ]:
#MASKS
masks_dir = ""#ADD PATH
destiny_path_masks_train = ""#ADD PATH
destiny_path_masks_test = ""#ADD PATH

train_test_split_gaofen_masks(masks_dir,destiny_path_masks_train,destiny_path_masks_test)

# Create Datasets

In [13]:
import os
import numpy as np
from PIL import Image
import cv2

def divide_image_into_quadrants(image, n):
    
    if(len(image.shape) == 3):
        rows, cols, _ = image.shape  
    else:
        rows, cols = image.shape  
    
    num_rows = int(n ** 0.5)
    num_cols = n // num_rows
    
    quadrant_rows = rows // num_rows
    quadrant_cols = cols // num_cols
    
    quadrants = []
    
    for i in range(num_rows):
        for j in range(num_cols):
            start_row = i * quadrant_rows
            end_row = start_row + quadrant_rows
            start_col = j * quadrant_cols
            end_col = start_col + quadrant_cols
            
            quadrant = image[start_row:end_row, start_col:end_col]
            quadrants.append(quadrant)
    
    return quadrants

def process_8bit_images(path_8bit_source, path_8bit_destiny):
    
    for image_name in os.listdir(path_8bit_source):
        print(f"{path_8bit_source}{image_name}")
        image8bits = Image.open(f"{path_8bit_source}{image_name}").convert('CMYK')
        B = 4
        multi = np.zeros((B, image8bits.size[1], image8bits.size[0]))

        for i in range (0, B):
            multi[i,:,:] = np.array(image8bits.getchannel(i))


        
        image_raw = np.moveaxis(multi, 0, -1)

        quadrants = divide_image_into_quadrants(image_raw, 4)


        for idx,q in enumerate(quadrants):  
            print(f"Saving image {image_name[:-4]}_{idx} with shape {q.shape}")
            newFile = open(f"{path_8bit_destiny}{os.path.splitext(image_name)[0]}_{idx}.rawb", "wb")
            q.tofile(newFile, format="%d")
            newFile.close()

def process_masks(path_source,path_destiny):
    for mask_name in os.listdir(path_source):
            img = cv2.imread(f'{path_source}{mask_name}',cv2.IMREAD_UNCHANGED)
            smaller_imgs = divide_image_into_quadrants(img,4)
            for idx, small_image in enumerate(smaller_imgs):
                filename = f'{path_destiny}{os.path.splitext(mask_name)[0]}'
                filename = filename[:-8]
                cv2.imwrite(f'{filename}_{idx}_24label.png',small_image)


In [ ]:
# TRAIN
path_8bit_train = "" #Path to the 8bit images
path_8bit_train_destiny = "" #Path to save the 8bit images in quadrants
process_8bit_images(path_8bit_train, path_8bit_train_destiny)

In [ ]:
# TEST
path_8bit_test = ""#Path to the 8bit images
path_8bit_test_destiny = ""#Path to save the 8bit images in quadrants
process_8bit_images(path_8bit_test, path_8bit_test_destiny)

In [ ]:
# TRAIN MASKS
path_masks_train = ""#Path to the masks
path_masks_train_destiny = ""#Path to save the masks in quadrants
process_masks(path_masks_train,path_masks_train_destiny)

In [ ]:
# TEST MASKS
path_masks_test = ""#Path to the masks
path_masks_test_destiny = ""#Path to save the masks in quadrants
process_masks(path_masks_test,path_masks_test_destiny)

# Read function

In [ ]:
def read_rawb_NirRGB(file_name):
    SMALL_IMAGE_SHAPE = (3454, 3650, 4)
    with open(file_name, 'rb') as file:
        img = file.read()
        
    img = np.frombuffer(img, dtype=np.float64)

    img = img.reshape(SMALL_IMAGE_SHAPE)

    return img.astype(np.uint8)

def read_rawb_RGB(file_name):
    img = read_rawb_NirRGB(file_name)
    
    return img[:, :, 1:4]  # Delete Nir channel